# Lecture 4

In [1]:
import numpy as np

### 4.1.3 Thought experiment

In [2]:
topic = {}
tfidf = dict(list(zip('cat dog apple lion NYC love'.split(), np.random.rand(6)))) 

This `tfidf` vector is just a random example, as if it were computed for a single document that contained these words in some random proportion.

In [5]:
tfidf

{'cat': 0.30743560800316083,
 'dog': 0.5168671642929766,
 'apple': 0.5555888696205626,
 'lion': 0.07329871754506678,
 'NYC': 0.38247893571977465,
 'love': 0.7274697256234035}

In [4]:
topic['pet'] = (  .3 * tfidf['cat'] 
                + .3 * tfidf['dog'] 
                + 0 * tfidf['apple']
                + 0 * tfidf['lion'] 
                - .2 * tfidf['NYC'] 
                + .2 * tfidf['love'])

topic['animal'] = (.1 * tfidf['cat'] 
                   + .1 * tfidf['dog'] 
                   - .1 * tfidf['apple']
                   + .5 * tfidf['lion'] 
                   + .1 * tfidf['NYC'] 
                   - .1 * tfidf['love'])

topic['city'] = (0 * tfidf['cat'] 
                 - .1 * tfidf['dog'] 
                 + .2 * tfidf['apple']
                 - .1 * tfidf['lion'] 
                 + .5 * tfidf['NYC'] 
                 + .1 * tfidf['love'])

*Hand-crafted* weights (.3, .3, 0, 0, -.2, .2) are multiplied by imaginary tfidf values to create topic vectors for your imaginary random document.

In [6]:
topic

{'pet': 0.316288989669567,
 'animal': 0.029021670049728,
 'city': 0.3160876261625359}

$\begin{bmatrix}
.3 & .3& 0& 0& -.2& .2\\
.1 &.1 &-.1& .5& .1 &-.1\\
0 &-.1& .2& -.1& .5 &.1\\
\end{bmatrix}$
$\begin{bmatrix}
\text{cat} \\ 
\text{dog} \\ 
\text{apple} \\ 
\text{lion} \\ 
\text{NYC} \\ 
\text{love} \\ 
\end{bmatrix}=
\begin{bmatrix}
\text{pet}\\
\text{animal}\\
\text{city}\\
\end{bmatrix}
$

**Go back to the slide.**

Topic to word? No problem, but the same problem still there.

How to assign the weights?

In [7]:
word_vector = {}
word_vector['cat'] = .3 * topic['pet'] + .1 * topic['animal'] + 0 * topic['city']
word_vector['dog'] = .3 * topic['pet'] + .1 * topic['animal'] - .1 * topic['city']
word_vector['apple'] = 0 * topic['pet'] - .1 * topic['animal'] + .2 * topic['city']
word_vector['lion'] = 0 * topic['pet'] + .5 * topic['animal'] - .1 * topic['city']
word_vector['NYC'] = -.2 * topic['pet'] + .1 * topic['animal'] + .5 * topic['city']
word_vector['love'] = .2 * topic['pet'] - .1 * topic['animal'] + .1 * topic['city']
word_vector

{'cat': 0.09778886390584289,
 'dog': 0.0661801012895893,
 'apple': 0.06031535822753438,
 'lion': -0.01709792759138959,
 'NYC': 0.09768818215232734,
 'love': 0.09196439354519419}

### 4.1.5 An LDA classifier

In [9]:
import pandas as pd
from nlpia.data.loaders import get_data
pd.options.display.width = 120
sms = get_data('sms-spam')

ModuleNotFoundError: No module named 'nlpia'